<a href="https://colab.research.google.com/github/akiragm/DeepLearnng/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import sentencepiece as spm

In [18]:


# 1. テキストデータを用意します（例えば、'sample_text.txt'という名前のファイルに保存）。
# このデータは、SentencePieceがサブワードモデルを学習するためのものです。
"""
sample_text.txtの内容:
これはテストテキストです。
SentencePieceは効果的なトークン化ツールです。
...
"""

# 2. SentencePieceモデルの学習
spm.SentencePieceTrainer.train('--input=/content/AI_traning_data.txt --model_prefix=m --vocab_size=2000')

# 3. 学習したモデルをロード
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# 4. テキストをトークン化
text = "Y4@DB4G56GA.00"
print(sp.encode_as_pieces(text))

['▁', 'Y', '4@', 'DB', '4', 'G', '56', 'GA', '.0', '0']


In [ ]:

# トークンIDへの変換
tokenized_text = sp.encode_as_ids(text)

# Transformerモデルの設定
d_model = 512  # 例として、モデルの次元を512とします
vocab_size = 2000  # 上で設定したvocab_size
max_seq_length = 100  # 例として、最大シーケンス長を100とします

# 3. Embedding
class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x)

embedding_layer = EmbeddingLayer(vocab_size, d_model)

# トークンIDをTensorに変換し、Embedding
input_tensor = torch.tensor(tokenized_text).unsqueeze(0)  # バッチの次元を追加
embedded = embedding_layer(input_tensor)

# 4. Transformerの利用
transformer_layer = nn.Transformer(d_model, nhead=8, num_encoder_layers=6)
output = transformer_layer(embedded, embedded)

# 5. タスク固有のヘッド (ここでは分類タスクを例としています)
class ClassificationHead(nn.Module):
    def __init__(self, d_model, num_classes):
        super(ClassificationHead, self).__init__()
        self.linear = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # ここで、[CLS]トークンとして最初のトークンを使用します
        return self.linear(x[:, 0, :])

num_classes = 10  # 例: 10クラス分類
classification_head = ClassificationHead(d_model, num_classes)
class_logits = classification_head(output)

print(class_logits)  # 分類タスクのロジットを表示
